In [1]:
DATE_GRAMMAR = """
?start: date

// Rule: A date can be in one of several formats
date: mdy_slash | mdy_text

// Format 1: MM/DD/YYYY or MM/DD
// The year and its preceding slash are now optional.
mdy_slash: MONTH_NUM "/" DAY_NUM ("/" YEAR)?

// Format 2: "Jan 15 2024" or "Jan 15"
// The year is now optional.
mdy_text: MONTH_TEXT DAY_NUM YEAR?


// === TERMINALS ===

MONTH_TEXT.2: /(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)/i
YEAR.1: /\\d{4}|\\d{2}/
MONTH_NUM.1: /1[0-2]|0?[1-9]/
DAY_NUM.1: /[12][0-9]|3[01]|0?[1-9]/


%import common.WS
%ignore WS
"""

In [2]:
from datetime import date, timedelta

from lark import Lark, Transformer, v_args

# ... (grammar string from above) ...
# ... (MONTH_MAP dictionary is unchanged) ...


class SemanticValidationError(ValueError):
    """Custom exception for semantic errors during parsing."""

    pass


class DateValidator(lark.Transformer):
    """
    This transformer now intelligently resolves the year if it is omitted,
    ensuring the resulting date is on or after today.
    """

    def _resolve_date(self, month, day, year=None):
        """Helper method to contain the core date resolution logic."""
        today = date.today()

        if year is None:
            # --- Year Inference Logic ---
            # 1. Create a candidate date with the current year.
            try:
                candidate_date = date(today.year, month, day)
            except ValueError as e:
                # This catches things like "Feb 29" in a non-leap year
                raise lark.LarkError(f"Invalid date: {e}")

            # 2. If the candidate date is in the past, use next year.
            if candidate_date < today:
                final_year = today.year + 1
            else:
                final_year = today.year
        else:
            # If year was provided, just use it.
            final_year = year

        # 3. Construct and return the final date object.
        try:
            return date(final_year, month, day)
        except ValueError as e:
            # Final validation for cases like "Nov 31 2024"
            raise lark.LarkError(f"Invalid date: {e}")

    # --- Methods to process tokens from the tree ---
    def YEAR(self, y):
        year_val = int(y)
        if year_val < 100:
            return 2000 + year_val
        return year_val

    def MONTH_NUM(self, m):
        return int(m)

    def DAY_NUM(self, d):
        return int(d)

    def MONTH_TEXT(self, m):
        return MONTH_MAP[m.upper()]

    # --- Methods to process rules from the tree ---
    def mdy_slash(self, items):
        # Unpack items; year might be missing
        month, day, *maybe_year = items
        year = maybe_year[0] if maybe_year else None
        return self._resolve_date(month, day, year)

    def mdy_text(self, items):
        # Same logic as mdy_slash
        month, day, *maybe_year = items
        year = maybe_year[0] if maybe_year else None
        return self._resolve_date(month, day, year)

    def date(self, items):
        return items[0]

NameError: name 'lark' is not defined

In [7]:
import inspect
from datetime import date, timedelta
from unittest.mock import patch  # We use patch to control "today's date"

import lark

DEBUG = False


def type_and_value(obj):  # pragma: no cover
    """Helper for debugging: returns the type and value of an object."""
    if DEBUG:  # pragma: no cover
        return f"{type(obj)}: {repr(obj)}"
    return ""  # pragma: no cover


# The updated grammar string with optional year
date_grammar = r"""
?start: date
date: mdy_slash | mdy_text
mdy_slash: MONTH_NUM "/" DAY_NUM ("/" YEAR)?
mdy_text: MONTH_TEXT DAY_NUM YEAR?
MONTH_TEXT.2: /(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)/i
YEAR.1: /\\d{4}|\\d{2}/
MONTH_NUM.1: /1[0-2]|0?[1-9]/
DAY_NUM.1: /[12][0-9]|3[01]|0?[1-9]/
%import common.WS
%ignore WS
"""

MONTH_MAP = {
    "JAN": 1,
    "FEB": 2,
    "MAR": 3,
    "APR": 4,
    "MAY": 5,
    "JUN": 6,
    "JUL": 7,
    "AUG": 8,
    "SEP": 9,
    "OCT": 10,
    "NOV": 11,
    "DEC": 12,
}


# The new Transformer class from above
class DateValidator(lark.Transformer):
    def _resolve_date(self, month, day, year=None):
        today = date.today()
        if year is None:
            try:
                candidate_date = date(today.year, month, day)
            except ValueError as e:
                raise lark.LarkError(f"Invalid date: {e}")
            if candidate_date < today:
                final_year = today.year + 1
            else:
                final_year = today.year
        else:
            final_year = year
        try:
            return date(final_year, month, day)
        except ValueError as e:
            raise lark.LarkError(f"Invalid date: {e}")

    def YEAR(self, y):
        if DEBUG:  # pragma: no cover
            print(f"{inspect.stack()[0][3]} {type_and_value(y)}")
        year_val = int(y)
        if year_val < 100:
            return 2000 + year_val
        return year_val

    def MONTH_NUM(self, m):
        return int(m)

    def DAY_NUM(self, d):
        return int(d)

    def MONTH_TEXT(self, m):
        return MONTH_MAP[m.upper()]

    def mdy_slash(self, items):
        month, day, *maybe_year = items
        year = maybe_year[0] if maybe_year else None
        return self._resolve_date(month, day, year)

    def mdy_text(self, items):
        month, day, *maybe_year = items
        year = maybe_year[0] if maybe_year else None
        return self._resolve_date(month, day, year)

    def date(self, items):
        return items[0]


def constraint_parser(grammar=DATE_GRAMMAR, debug=False):
    constraint_transformer = DateValidator()
    global DEBUG
    DEBUG = debug
    return Lark(grammar, parser="lalr", transformer=constraint_transformer, debug=debug)

In [8]:
parser = constraint_parser(debug=True)

In [13]:
result = parser.parse("Jan 2")
print(f"\nres = {result}")


res = 2027-01-02


In [ ]:
# --- Let's Test It ---
parser = lark.Lark(DATE_GRAMMAR)
validator = DateValidator()

# Let's PRETEND today is April 15, 2024 for consistent testing
MOCK_TODAY = date(2024, 4, 15)

# Test cases
test_dates = {
    # --- Year is provided ---
    "10/25/2025": date(2025, 10, 25),
    "FEB 29 2024": date(2024, 2, 29),
    # --- Year is omitted, date is in the future this year ---
    "12/25": date(2024, 12, 25),
    "MAY 1": date(2024, 5, 1),
    # --- Year is omitted, date is in the past this year (should roll over) ---
    "1/15": date(2025, 1, 15),
    "MAR 10": date(2025, 3, 10),
    # --- Edge case: today's date ---
    "4/15": date(2024, 4, 15),
}

print(f"--- Testing with MOCK_TODAY = {MOCK_TODAY} ---")

# We use the "patch" context manager to temporarily change what date.today() returns
with patch("__main__.date") as mock_date:
    mock_date.today.return_value = MOCK_TODAY
    mock_date.side_effect = lambda *args, **kw: date(*args, **kw)

    for text, expected in test_dates.items():
        try:
            tree = parser.parse(text)
            result = validator.transform(tree)
            status = "✅" if result == expected else "❌"
            print(f'{status} "{text}" -> Resolved to: {result} (Expected: {expected})')
        except lark.LarkError as e:
            print(f'"{text}" -> FAILED: {e}')